<a href="https://colab.research.google.com/github/GianRomani/Neural_search_engine/blob/main/DM_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try: 
  #this will raise an error if you are not using Colab
  from google.colab import drive
  drive.mount('/content/drive')
except:
  print("You are not using Colab")

Mounted at /content/drive


In [2]:
path = "/content/drive/MyDrive/Data Mining/Project/archive.zip"

In [3]:
#!unzip "/content/drive/MyDrive/Data Mining/Project/archive.zip" -d "/content/drive/MyDrive/Data Mining/Project/"

# EDA

In [4]:
import pandas as pd

It can be interesting and helpful to analyze:

*   the distribution of the scores among the questions,
*   number of answers for question (and scores of those answers);
*   creation dates for the posts;
*   if a post was closed (and why);
*   popular users (who answered a lot of questions)





<h2>Questions</h2>

*   Id: id of the question;
*   OwnerUserId: id of the account who post the question;
*   CreationDate: when the question was posted;
*   ClosedDate: when the question was closed;
*   Score: upvotes - downvotes;
*   Title: title of the question;
*   Body: the question

In [5]:
questions = pd.read_csv("/content/drive/MyDrive/Data Mining/Project/Questions.csv",encoding="ISO-8859-1") #utf-8 is not good in this case

In [6]:
questions.head(10)

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...
5,330,63.0,2008-08-02T02:51:36Z,NaN,29,Should I use nested classes in this case?,<p>I am working on a collection of classes use...
6,470,71.0,2008-08-02T15:11:47Z,2016-03-26T05:23:29Z,13,Homegrown consumption of web services,<p>I've been writing a few web services for a ...
7,580,91.0,2008-08-02T23:30:59Z,NaN,21,Deploying SQL Server Databases from Test to Live,<p>I wonder how you guys manage deployment of ...
8,650,143.0,2008-08-03T11:12:52Z,NaN,79,Automatically update version number,<p>I would like the version property of my app...
9,810,233.0,2008-08-03T20:35:01Z,NaN,9,Visual Studio Setup Project - Per User Registr...,<p>I'm trying to maintain a Setup Project in <...


In [7]:
questions.describe()

,Id,OwnerUserId,Score
count,1.264216e+06,1.249762e+06,1.264216e+06
mean,2.132745e+07,2.155177e+06,1.781537e+00
std,1.151445e+07,1.801265e+06,1.366389e+01
min,8.000000e+01,1.000000e+00,-7.300000e+01
25%,1.142598e+07,6.589110e+05,0.000000e+00
50%,2.172542e+07,1.611830e+06,0.000000e+00
75%,3.154542e+07,3.353792e+06,1.000000e+00
max,4.014338e+07,7.046594e+06,5.190000e+03


In [8]:
#What is the distribution of the scores among the questions?
score_distribution = questions["Score"].value_counts(bins=20)
score_distribution

(-78.26400000000001, 190.15]    1263601
(190.15, 453.3]                     478
(453.3, 716.45]                      78
(716.45, 979.6]                      30
(979.6, 1242.75]                     11
(1505.9, 1769.05]                     9
(1242.75, 1505.9]                     4
(2295.35, 2558.5]                     3
(3611.1, 3874.25]                     1
(4926.85, 5190.0]                     1
(2821.65, 3084.8]                     0
(2558.5, 2821.65]                     0
(4663.7, 4926.85]                     0
(2032.2, 2295.35]                     0
(1769.05, 2032.2]                     0
(3084.8, 3347.95]                     0
(3347.95, 3611.1]                     0
(3874.25, 4137.4]                     0
(4137.4, 4400.55]                     0
(4400.55, 4663.7]                     0
Name: Score, dtype: int64

In [11]:
#negative scores
negative_scores_questions = questions[questions["Score"] < 0]
negative_scores_questions

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
487,43960,3262.0,2008-09-04T15:07:30Z,NaN,-1,COTS Workshop Registration System,<p>Does anyone have any experience with any CO...
890,73260,12550.0,2008-09-16T14:57:37Z,NaN,-1,What is your choice for a Time Managment Solut...,<p>I've come across a few different applicatio...
904,74570,7709.0,2008-09-16T17:02:14Z,NaN,-1,CSS : Bad Gray Line to the side of the Navigat...,"<p>I'm maintaining <a href=""http://perl-begin...."
949,79350,NaN,2008-09-17T02:37:37Z,2015-06-23T23:33:20Z,-1,What open source virtual private server progra...,<p>I am looking to have 4 Virtual servers(vari...
1017,85370,6992.0,2008-09-17T17:01:08Z,NaN,-2,"Most common cause of ""java.lang.NullPointerExc...",<p>My strongest lead is that the code who deal...
...,...,...,...,...,...,...,...
1264187,40142600,7044841.0,2016-10-19T22:33:00Z,2016-10-19T22:34:48Z,-10,Remove white spaces without replace(),<pre><code>def compress(s):\n âââ(str...
1264188,40142610,7037290.0,2016-10-19T22:34:23Z,NaN,-1,"How to get a direct, secure url (XXX.mp3) for ...",<p>So I can pull the url for a track from the ...
1264193,40142780,4957922.0,2016-10-19T22:52:04Z,NaN,-4,Sparse Arrays in Data Structure,<p>I am solving the following problem in C : \...
1264194,40142790,7044882.0,2016-10-19T22:52:53Z,2016-10-19T23:05:09Z,-5,Why is my switch statement giving me constant ...,<p>Sorry in advance for the weird format of th...


In [12]:
#Posts with a score greater than 4000
best_posts = questions[questions["Score"] >4000]
best_posts

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
4907,348170,14069.0,2008-12-07T21:57:46Z,NaN,5190,How to undo 'git add' before commit?,<p>I mistakenly added files using the command:...


In [13]:
#Count NaN values -> most of them in ClosedDate
questions.isna().sum()

Id                    0
OwnerUserId       14454
CreationDate          0
ClosedDate      1208257
Score                 0
Title                 0
Body                  0
dtype: int64

<h2>Answers</h2>



*   Id: id of the answer;
*   OwnerUserId: id of the account who post the answer;
*   CreationDate: when the answer was posted;
*   ParentId: Id of the question;
*   Score: upvotes - downvotes;
*   Body: the answer



In [14]:
answers = pd.read_csv("/content/drive/MyDrive/Data Mining/Project/Answers.csv",encoding="ISO-8859-1") #utf-8 is not good in this case

In [15]:
answers.head(10)

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
1,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...
2,199,50.0,2008-08-01T19:36:46Z,180,1,<p>I've read somewhere the human eye can't dis...
3,269,91.0,2008-08-01T23:49:57Z,260,4,"<p>Yes, I thought about that, but I soon figur..."
4,307,49.0,2008-08-02T01:49:46Z,260,28,"<p><a href=""http://www.codeproject.com/Article..."
5,332,59.0,2008-08-02T03:00:24Z,330,19,<p>I would be a bit reluctant to use nested cl...
6,344,100.0,2008-08-02T04:18:15Z,260,6,<p>You might be able to use IronRuby for that....
7,359,119.0,2008-08-02T06:16:23Z,260,5,"<P>You could use any of the DLR languages, whi..."
8,473,49.0,2008-08-02T15:33:13Z,470,8,"<p>No, what you're doing is fine. Don't let th..."
9,529,86.0,2008-08-02T18:16:07Z,180,3,<p>Isn't it also a factor which order you set ...


In [16]:
answers.describe()

,Id,OwnerUserId,ParentId,Score
count,2.014516e+06,2.001316e+06,2.014516e+06,2.014516e+06
mean,1.915490e+07,1.487613e+06,1.808390e+07,2.480563e+00
std,1.168713e+07,1.549051e+06,1.169152e+07,1.590938e+01
min,9.200000e+01,1.000000e+00,8.000000e+01,-4.200000e+01
25%,8.854490e+06,2.818680e+05,7.692900e+06,0.000000e+00
50%,1.866246e+07,9.546430e+05,1.712404e+07,1.000000e+00
75%,2.929756e+07,2.197072e+06,2.804977e+07,2.000000e+00
max,4.014339e+07,7.045028e+06,4.014319e+07,5.718000e+03


In [17]:
#Distribution for the scores of the answers
score_distribution = answers["Score"].value_counts(bins=20)
score_distribution

(-47.760999999999996, 246.0]    2013578
(246.0, 534.0]                      708
(534.0, 822.0]                      150
(822.0, 1110.0]                      34
(1398.0, 1686.0]                     17
(1110.0, 1398.0]                      8
(1686.0, 1974.0]                      6
(2262.0, 2550.0]                      4
(2838.0, 3126.0]                      4
(1974.0, 2262.0]                      3
(3414.0, 3702.0]                      2
(5430.0, 5718.0]                      1
(2550.0, 2838.0]                      1
(5142.0, 5430.0]                      0
(3126.0, 3414.0]                      0
(3702.0, 3990.0]                      0
(3990.0, 4278.0]                      0
(4278.0, 4566.0]                      0
(4566.0, 4854.0]                      0
(4854.0, 5142.0]                      0
Name: Score, dtype: int64

In [27]:
#negative scores
negative_scores_answers = answers[answers["Score"] < 0]
negative_scores_answers

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
259,7896,268.0,2008-08-11T16:07:31Z,7880,-2,<p>fstream are great but I will go a little de...
284,8421,785.0,2008-08-12T02:37:15Z,6430,-2,"<p>Aha, I was really just testing everyone onc..."
386,10687,234.0,2008-08-14T05:03:34Z,10610,-1,<p>e-texteditor hilights what you're searching...
483,12098,198.0,2008-08-15T09:24:51Z,10980,-2,<p>I'm not a lawyer but all licenses mentioned...
498,12885,872.0,2008-08-15T23:27:13Z,12870,-1,<p>There is no pretty solution. Java just doe...
...,...,...,...,...,...,...
2014340,40139542,541038.0,2016-10-19T19:02:35Z,40139460,-1,<pre><code>CHUNKSIZE=1024 # read 1024 bytes at...
2014343,40139568,3220135.0,2016-10-19T19:04:27Z,40139460,-1,<p>depending on how your particular data is br...
2014347,40139589,5912548.0,2016-10-19T19:06:08Z,40137060,-1,"<p>I don't have ggplot installed, but maybe a ..."
2014360,40139866,3831435.0,2016-10-19T19:21:27Z,40139500,-1,"<p>Try this:</p>\n\n<pre><code>#include ""stdaf..."


In [18]:
#Number of answers for question
answers_for_post = answers.groupby(["ParentId"]).count()["Id"]
answers_for_post

ParentId
80          3
90          3
120         1
180         9
260         9
           ..
40142860    1
40142900    1
40142910    1
40142940    2
40143190    2
Name: Id, Length: 1102568, dtype: int64

In [26]:
#max number of answers for a post
answers_for_post.max()

408

In [25]:
#How many questions have a certain number of answers?
answers_for_post.value_counts()

1      606272
2      284868
3      117396
4       49255
5       21767
6       10005
7        5132
8        2760
9        1553
10        954
11        702
12        508
13        311
14        237
15        187
16        125
17        100
18         86
19         54
20         42
21         31
22         31
23         29
24         22
25         17
27         16
26         15
29         12
28         11
31         11
30         11
32          7
33          5
34          5
38          3
36          3
40          3
43          2
35          2
39          2
61          1
67          1
69          1
316         1
100         1
55          1
129         1
59          1
42          1
51          1
50          1
49          1
45          1
41          1
37          1
408         1
Name: Id, dtype: int64

In [20]:
#How may NaN values?
answers.isna().sum()

Id                  0
OwnerUserId     13200
CreationDate        0
ParentId            0
Score               0
Body                0
dtype: int64

In [ ]:
#How many users answered questions?


<h2>Tags</h2>